In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from IPython.display import clear_output

# Collect list of countries

In [3]:
url = 'https://www.officeholidays.com/countries'

r = requests.get(url)
html = r.text
soup = BeautifulSoup(html)
columns = soup.find_all('div', {"class": "four omega columns"})
countries = []
for column in columns:
    rows = column.find_all('li')
    for row in rows:
        countries.append(row.find('a').text[2:])

In [55]:
countries_processed = []

for country in countries:
    country = country.replace(' ', '-')
    country = country.replace('é', 'e')
    countries_processed.append(country.lower())

# Testing if all countries have links correctly matching their overall form

In [67]:
for country in countries_processed:
    r = requests.get(url+'/'+country)
    if r.status_code != 200:
        print(country)

Found issues on four countries 
* bonaire,-sint-eustatius-and-saba
* eswatini
* north-macedonia
* united-arab-emirates

Fixing manually in list

In [69]:
#for index, country in enumerate(countries_processed):
    #print(index, country)
    
countries_processed[25] = 'bonaire-st-eustatius-saba'
countries_processed[64] = 'swaziland'
countries_processed[149] = 'macedonia'
countries_processed[208] = 'uae'

# Final data collection

In [149]:
def convert_month(txt):
    dictionary = {'jan': '01', 'feb': '02', 'mar': '03', 'apr': '04',
                  'may': '05', 'jun': '06', 'jul': '07', 'aug': '08',
                  'sep': '09', 'oct': '10', 'nov': '11', 'dec': '12'}
    
    for k, v in dictionary.items():
        if txt.lower() == k:
            return dictionary[k]

In [242]:
df = pd.DataFrame()
df_tmp = pd.DataFrame()
years = list(range(2016,2023))

for index, country in enumerate(countries_processed):
    for year in years:
        count = 0
        data = []
        tmp_url = url + '/' + country + '/' + str(year)
        
        r = requests.get(tmp_url)
        
        if r.status_code == 200:
            html = r.text
            soup = BeautifulSoup(html)
            table = soup.find('table', {'class': 'country-table'})
            if table:
                rows = table.find_all('tr')
                for row in rows[1:]:
                    cols = row.find_all('td')
                    cols = [ele.text.strip() for ele in cols]
                    data.append([ele for ele in cols if ele])
                    
                    count += 1

                df_tmp = pd.DataFrame(data)

                for i, row in df_tmp.iterrows():
                    tmp = pd.Series([str(year)+'-'+convert_month(row[1][:3])+'-'+row[1][-2:], row[2], countries[index]])
                    df = df.append(tmp, ignore_index=True)
                    
        print('Latest processed: ' + country + '. holidays: ' + str(count))
        print(tmp_url)
            
    clear_output(wait=True)

Latest processed: american-samoa. holidays: 0
https://www.officeholidays.com/countries/american-samoa/2016
Latest processed: american-samoa. holidays: 0
https://www.officeholidays.com/countries/american-samoa/2017
Latest processed: american-samoa. holidays: 0
https://www.officeholidays.com/countries/american-samoa/2018
Latest processed: american-samoa. holidays: 14
https://www.officeholidays.com/countries/american-samoa/2019


KeyboardInterrupt: 

In [244]:
df

,0,1,2
0,2018-01-29,Public Holiday,Afghanistan
1,2018-02-15,Liberation Day,Afghanistan
2,2018-02-28,Public Holiday,Afghanistan
3,2018-03-21,Nowrooz,Afghanistan
4,2018-03-22,Nowrooz Holiday,Afghanistan
...,...,...,...
281,2019-10-13,White Sunday,American Samoa
282,2019-10-14,Columbus Day,American Samoa
283,2019-11-11,Veterans Day,American Samoa
284,2019-11-28,Thanksgiving,American Samoa


In [238]:
df.columns = ['date', 'holiday', 'country']

In [ ]:
df.to_csv('holiday_data.csv', index=False)